In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline, EulerDiscreteScheduler
from huggingface_hub import hf_hub_download

base = "stabilityai/stable-diffusion-xl-base-1.0"
repo = "ByteDance/SDXL-Lightning"
ckpt = "sdxl_lightning_4step_lora.safetensors"

pixel_lora = "nerijs/pixel-art-xl"

# Load model.
pipe = StableDiffusionXLPipeline.from_pretrained(base, torch_dtype=torch.float16, variant="fp16")
pipe.load_lora_weights(hf_hub_download(repo, ckpt), adapter_name="lightning")
pipe.load_lora_weights(pixel_lora, adapter_name="pixel")
pipe.set_adapters(["lightning", "pixel"], adapter_weights=[1.0, 1.2])
pipe.fuse_lora()

# Ensure sampler uses "trailing" timesteps.
pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config, timestep_spacing="trailing")
pipe.enable_model_cpu_offload()


In [ ]:
from datetime import datetime
from pixeldetector import main
import os

prompt = "Colorful sunset behind a mountain range with a lake in the foreground. Award winning. Landscape."
negative_prompt = "text, watermark, blurry, deformed, depth of field, realistic. 3d render, outline, border"
num_images = 1
base_directory = "img"
os.makedirs(base_directory, exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d-%H%M%S")
for i in range(num_images):
    img = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=4,
        guidance_scale=2,
    ).images[0]
    
    img_base = f"{base_directory}/{timestamp}_lightning_{i}"
    img_path = f"{img_base}.png"
    img.save(img_path)

    repaired_path = f"{img_base}_repaired.png"
    main(img_path, repaired_path)